In [5]:
# Exercise 2

import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.linalg import expm, eig
from numba import jit
from scipy.linalg import expm, eig
from scipy.special import comb
from qiskit import QuantumCircuit


In [6]:
N = 2
S = float(N)/2
Omega = np.pi*0.5
g = np.pi*0.5
mJ = np.arange(-S, S+1, dtype=float)
Jz2 = np.diag(np.square(mJ)).astype(complex)
Jz = np.diag(mJ).astype(complex)
Jp = np.diag(np.sqrt(S*(S+1) - mJ[:-1]*mJ[1:]), -1).astype(complex)
Jm = np.diag(np.sqrt(S*(S+1) - mJ[1:]*mJ[:-1]), 1).astype(complex)
Rx = expm(-1j*Omega*(Jp + Jm)/2)
Ry = expm(-Omega*(Jp - Jm)/2)
Sq = expm(-1j*g*Jz2)
psi0 = np.zeros(N+1, dtype=complex)
psi0[0] = 1.

n_theta = 5
n_phi = 4

CSS = [[np.zeros(N+1, dtype=complex)]*n_phi for i in range(n_theta)]
for i in range(n_theta):
    for j in range(n_phi):
        theta = i * np.pi / (n_theta - 1)
        phi = j * 2*np.pi / n_phi - np.pi
        css = np.zeros(N+1, dtype=complex)
        for k in range(N+1):
            css[k] = np.sqrt(comb(N, k)) * np.cos(theta/2)**(N-k) * np.sin(theta/2)**k
            css[k] *= np.exp(-1j*k*phi);
        CSS[i][j] = css
        #print(i, j, CSS[i][j])

#print(CSS)
class Env(object):
    
    def __init__(self, Omega, var=0.00):
        super(Env, self).__init__()
        self.Omega = Omega
        self.n_actions = 3
        self.actions = [Rx, Ry, Sq]
        self.n_states = n_theta * n_phi #N+1
        #self.Ut = np.identity(n_states, dtype=complex)
        #self.state = np.square(np.absolute(np.concatenate((self.Ut[:,1], self.Ut[:,5]))))
        self.psi = psi0
        self.state = self.husimiQ()#np.square(np.absolute(self.psi))
        self.n_steps = 10
        self.t = 0
        self.threshold = 0.95
        #self.noise = 0
        #self.var = var
        
        print(Sq)#np.conjugate(Rx.T).dot(Rx))
        print(Rx)#np.conjugate(Ry.T).dot(Ry))
        print(Ry)#np.conjugate(Sq.T).dot(Sq))
        print(self.state)
        
    def reset(self):
        self.psi = psi0
        self.state = self.husimiQ()#np.square(np.absolute(self.psi))
        self.t = 0
        #self.noise = 0

        return self.state

    def step(self, action):
        U = self.actions[action]
        self.psi = U.dot(self.psi)
        QFI = np.einsum('i,ij,j->', np.conjugate(self.psi), Jz2, self.psi) - np.abs(np.einsum('i,ij,j->', np.conjugate(self.psi), Jz, self.psi))**2
        QFI = np.real(QFI)
        QFI *= 4
        QFI /= N**2
        #reward
        done =( QFI > self.threshold or self.t >= self.n_steps-1)
        rwd = done * QFI
        if QFI > self.threshold:
            print(self.psi)
        self.t += 1
        self.state = self.husimiQ()#np.square(np.absolute(self.psi))
        #self.state = np.square(np.absolute(np.concatenate((self.Ut[:,1], self.Ut[:,5])))) 
        return self.state, rwd, done, QFI
        
    def husimiQ(self):
        Q = np.zeros([n_theta, n_phi])
        for i in range(n_theta):
            for j in range(n_phi):
                Q[i][j] = abs(np.conjugate(CSS[i][j]).dot(self.psi)) ** 2
        return Q.reshape(n_theta * n_phi)

In [7]:
class QuantumAgent(object):
    def __init__(self, env, learning_rate):
        super(QuantumAgent, self).__init__()
        self.env = env
        self.n_actions = env.n_actions
        self.current_state = None
        self.episode_actions = []  # Actions taken in the current episode
        self.policy = np.ones(self.n_actions) / self.n_actions  # Initialize the policy uniformly
        self.learning_rate = learning_rate  # Set the learning rate
        
    def select_action(self):
        # Sample an action from the current policy
        return np.random.choice(self.n_actions, p=self.policy)
    
    def update_policy(self, qfi):
        # Update the policy using the Grover algorithm
        self.policy = self.grover_update(self.policy, qfi)
        
        # Normalize the policy distribution
        self.policy = self.policy / np.sum(self.policy)
    
    def train(self, num_episodes):
        for episode in range(num_episodes):
            self.current_state = self.env.reset()
            self.episode_actions = []  # Reset the actions for each episode
            done = False
            
            while not done:
                action = self.select_action()
                next_state, reward, done, qfi = self.env.step(action)
                
                # Store the actions taken in the current episode
                self.episode_actions.append(action)
                
                self.current_state = next_state
            
            # Update the policy based on the QFI for the episode
            self.update_policy(qfi)
            
            # Print the QFI for the episode
            print("Episode:", episode, "QFI:", qfi)
            
            # Generate and print the quantum circuit for the episode
            qc = self.generate_quantum_circuit(self.episode_actions)
            print(qc)
            
    def generate_quantum_circuit(self, episode_actions):
        # Generate a quantum circuit based on the actions taken in the episode
        r= random.randint(0, 1)
        qc = QuantumCircuit(2)
        for action in episode_actions:
            # Apply the corresponding gate based on the action
            if action == 0:
                qc.rx(self.env.Omega, range(2))  # Apply Rx gate to all qubits
            elif action == 1:
                qc.ry(self.env.Omega, range(2))  # Apply Ry gate to all qubits
            elif action == 2:
                 qc.s(r)  # Apply S gate only to the first qubit

        return qc
    
    def grover_update(self, policy, qfi):
        # Calculate the Grover amplitude update
        grover_update = 2 * qfi - 1
        
        # Apply the update to the policy
        updated_policy = policy + self.learning_rate * grover_update
        
        # Ensure that the updated policy probabilities remain non-negative
        updated_policy = np.maximum(updated_policy, 0)
        
        return updated_policy


# Create an instance of the environment
env = Env(Omega=np.pi * 0.5)

# Set the learning rate for the agent
learning_rate = 0.1

# Create an instance of the quantum agent
agent = QuantumAgent(env, learning_rate)

# Train the agent
agent.train(num_episodes=10)

[[6.123234e-17-1.j 0.000000e+00+0.j 0.000000e+00+0.j]
 [0.000000e+00+0.j 1.000000e+00+0.j 0.000000e+00+0.j]
 [0.000000e+00+0.j 0.000000e+00+0.j 6.123234e-17-1.j]]
[[ 0.5+0.j          0. -0.70710678j -0.5+0.j        ]
 [ 0. -0.70710678j  0. +0.j          0. -0.70710678j]
 [-0.5+0.j          0. -0.70710678j  0.5+0.j        ]]
[[ 0.5       +0.j  0.70710678+0.j  0.5       +0.j]
 [-0.70710678+0.j  0.        +0.j  0.70710678+0.j]
 [ 0.5       +0.j -0.70710678+0.j  0.5       +0.j]]
[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 7.28553391e-01 7.28553391e-01 7.28553391e-01 7.28553391e-01
 2.50000000e-01 2.50000000e-01 2.50000000e-01 2.50000000e-01
 2.14466094e-02 2.14466094e-02 2.14466094e-02 2.14466094e-02
 1.40579963e-65 1.40579963e-65 1.40579963e-65 1.40579963e-65]
Episode: 0 QFI: 0.0
     ┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐     »
q_0: ┤ Rx(π/2) ├┤ Rx(π/2) ├┤ Rx(π/2) ├┤ Ry(π/2) ├┤ Ry(π/2) ├┤ Rx(π/2) ├─────»
     ├─────────┤└──┬───┬──┘└──┬───┬──┘